In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import pandas as pd
import urllib.request as request

In [6]:
# Download Data
data_url="https://download.pytorch.org/tutorial/hymenoptera_data.zip"

In [3]:
class Config:
    def __init__(self):
        self.ROOT_DATA_DIR="Image_Data_Dir"
        self.EPOCH=100
        self.BATCH_SIZE=32
        self.LEARNING_RATE=0.001
        #self.IMAGE_SIZE=(28,28)
        self.DEVICE="cuda" if torch.cuda.is_available() else "cpu"
        print(f"This notebook is using the device: {self.DEVICE}")
        self.SEED=2022
        
    def create_dir(self, dir_path):
        os.makedirs(dir_path, exist_ok=True)
        print(f"{dir_path} directory is created")
config=Config()

This notebook is using the device: cpu


In [4]:
config.create_dir(dir_path=config.ROOT_DATA_DIR)

Image_Data_Dir directory is created


In [5]:
data_zip_file="data.zip"

In [7]:
data_file_path=os.path.join(config.ROOT_DATA_DIR, data_zip_file)
request.urlretrieve(data_url, data_file_path)

('Image_Data_Dir\\data.zip', <http.client.HTTPMessage at 0x29081c59700>)

In [10]:
from zipfile import ZipFile
def unzip_file(source: str, dest: str) -> None:
    print(f"extraction started.....")
    
    with ZipFile(source, "r") as zip_f:
        zip_f.extractall(dest)
    print(f"extracted {source} to {dest}")

In [11]:
unzip_file(data_file_path, config.ROOT_DATA_DIR)

extraction started.....
extracted Image_Data_Dir\data.zip to Image_Data_Dir


In [13]:
# Create Data Loader
!ls Image_Data_Dir/hymenoptera_data/train

ants
bees
